In [32]:
import pandas as pd
import os
from Bio import Seq
import re

%cd /Users/thiberio/t3ss_effectors/signal_prediction/

/Users/thiberio/t3ss_effectors/signal_prediction


In [23]:
for filename in os.listdir('.'):
    if not filename.endswith('.csv'):
        continue
    df = pd.read_table(filename, sep='\t')
    tmp_df = df.query('~protein.str.startswith("gi")').copy()
    print(filename,
          tmp_df.shape[0],
          sum(tmp_df['Search for secreted type-III proteins (EffectiveT3)'].astype(float) >= 0.95))

PteA.csv 27 20
PteB.csv 30 15
AopO.csv 5 5
AopX.csv 36 36
PteC.csv 5 5
PteG.csv 7 5
PteF.csv 3 2
AopH.csv 9 1
PteE.csv 51 4
AexU.csv 42 24
AexT.csv 20 9
AopS.csv 5 0
PteI.csv 2 0
aopP.csv 6 6
Ati2.csv 6 6
PteJ.csv 1 1
PteM.csv 2 1
pteL.csv 3 3


In [30]:
for protein in 'PteD PteD.1 PteH PteK PteM.1'.split():
    fasta = open('../%s.fna' % protein).readlines()
    out   = open('../%s.faa' % protein, 'w')
    for line in fasta:
        if line.startswith('>'):
            out.write(line)
        else:
            out.write(Seq.translate(line))
            out.write('\n')
    out.close()

In [129]:
%cd /Users/thiberio/t3ss_effectors/
chaperone_files = {}
for filename in os.listdir('.'):
    if not filename.endswith('.fna') or re.match('\d', filename):
        continue

    effectors = {}
    for seq_id, genome in re.findall('^>(\S+?)\|(\S+)$', open(filename).read(), re.M):
        if 'contig' in genome  or '[' in genome:
            continue
        if not genome in effectors:
            effectors[genome] = []
        effectors[genome].append(seq_id)
    
    downstream_names = {}
    for genome, seqs in effectors.items():
        if not genome in downstream_names:
            downstream_names[genome] = []

        for effector in seqs:
            seq_id_fragments = effector.split('.')
            downstream_names[genome].append('%s.%i' %('.'.join(seq_id_fragments[:-1]),
                                            int(seq_id_fragments[-1])-1))
            downstream_names[genome].append('%s.%i' %('.'.join(seq_id_fragments[:-1]),
                                            int(seq_id_fragments[-1])+1))

    count = 0
    for filename2 in os.listdir('all_fastas/'):
        if filename2.startswith('.') or not filename2.endswith('.fna'):
            continue

        candidate_chaperones = {}
        for seq_id, genome in re.findall('^>(\S+?)\|(\S+)$', open('all_fastas/%s' % filename2).read(), re.M):
            if not genome in candidate_chaperones:
                candidate_chaperones[genome] = []
            candidate_chaperones[genome].append(seq_id)

        adjacent_to_effector = []
        for genome in effectors.keys():
            if genome not in candidate_chaperones:
                continue

            if set(downstream_names[genome]).intersection(candidate_chaperones[genome]):
                adjacent_to_effector.append(genome)
                
        if adjacent_to_effector:
            count += 1
            print(filename, filename2)
            if filename2 not in chaperone_files:
                chaperone_files[filename2] = []
            chaperone_files[filename2].append(filename)
#            os.system('cp all_fastas/%s possible_chaperones/associated_to_%s-%i.fna' % 
#                     (filename2,
#                      filename.replace('.fna', ''),
#                      count))

/Users/thiberio/t3ss_effectors
PteL.fna 723_642.148.peg.725.fna
Ati2.fna 233144_42949.4185.fna
PteK.fna 1897_642.148.peg.1899.fna
AexT.fna 320085_93602.2233-333379_41.3491.fna
AexT.fna 24318_35494.3581-320086_93602.2234-7480_50.3401.fna
AexU.fna 24318_35494.3581-320086_93602.2234-7480_50.3401.fna
AexU.fna 253597_642.133.peg.2831-253599_642.133.peg.2833-7479_50.3400-253600_642.133.peg.2834-253596_642.133.peg.2830-256111_642.134.peg.949.fna
PteD.fna 198567_18299.3086.fna
PteD.fna 333444_41.3556-320672_93602.2940.fna
AopH.fna 1897_642.148.peg.1899.fna
PteG.fna 1849_642.148.peg.1851-331672_41.1784.fna
PteG.fna 75867_26798.3983-70960_26797.2786.fna
AopX.fna 320663_93602.2927-333783_41.3895.fna
PteC.fna 70974_26797.2800.fna
PteC.fna 70972_26797.2798.fna
PteB.fna 321038_93602.2081.fna
PteD.1.fna 198567_18299.3086.fna
PteA.fna 320899_93602.957.fna


In [137]:
for filename, effectors in chaperone_files.items():
    associated_to = '_'.join([effector.replace('.fna', '') for effector in effectors])
    count = 0
    while True:
        count   += 1
        new_name = 'associated_to_%s-%i.fna' % (associated_to, count)
        if not os.path.isfile('possible_chaperones/%s' % new_name):
            break
            
    os.system('cp all_fastas/%s possible_chaperones/%s' % (filename, new_name))

In [134]:
chaperone_files

{'723_642.148.peg.725.fna': ['PteL.fna'],
 '233144_42949.4185.fna': ['Ati2.fna'],
 '1897_642.148.peg.1899.fna': ['PteK.fna', 'AopH.fna'],
 '320085_93602.2233-333379_41.3491.fna': ['AexT.fna'],
 '24318_35494.3581-320086_93602.2234-7480_50.3401.fna': ['AexT.fna',
  'AexU.fna'],
 '253597_642.133.peg.2831-253599_642.133.peg.2833-7479_50.3400-253600_642.133.peg.2834-253596_642.133.peg.2830-256111_642.134.peg.949.fna': ['AexU.fna'],
 '198567_18299.3086.fna': ['PteD.fna', 'PteD.1.fna'],
 '333444_41.3556-320672_93602.2940.fna': ['PteD.fna'],
 '1849_642.148.peg.1851-331672_41.1784.fna': ['PteG.fna'],
 '75867_26798.3983-70960_26797.2786.fna': ['PteG.fna'],
 '320663_93602.2927-333783_41.3895.fna': ['AopX.fna'],
 '70974_26797.2800.fna': ['PteC.fna'],
 '70972_26797.2798.fna': ['PteC.fna'],
 '321038_93602.2081.fna': ['PteB.fna'],
 '320899_93602.957.fna': ['PteA.fna']}

In [138]:
for filename in os.listdir('possible_chaperones/'):
    fasta = open('possible_chaperones/%s' % filename).readlines()
    out   = open('possible_chaperones/%s.faa' % filename.replace('.fna', ''), 'w')
    for line in fasta:
        if line.startswith('>'):
            out.write(line)
        else:
            out.write(Seq.translate(line))
            out.write('\n')
    out.close()

/Users/thiberio/anaconda2/envs/py37/lib/python3.7/site-packages/Bio/Seq.py:2576: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning)


In [102]:
for genome in downstream_names.keys():
    if genome in candidate_chaperones:
        print(genome, effectors[genome], candidate_chaperones[genome])

A_ichthiosmia_CECT4486 ['26530.1869'] ['26530.1867']
A_veronii_AMC25 ['26537.3590'] ['26537.3588']
A_veronii_AK247 ['31.2697'] ['31.2695']
A_veronii_G3C1 ['32.2789'] ['32.2790']
A_culicicola_CIP107763 ['36.1673'] ['36.1675']
A_allosaccharophila_BVH88 ['50.3400'] ['50.3402']
A_veronii_CECT7059 ['51.2150'] ['51.2149']
A_veronii_AK241 ['57.923'] ['57.924']
A_veronii_CECT4902 ['58.1275'] ['58.1274']
A_veronii_BVH37 ['642.149.peg.382'] ['642.149.peg.380']
A_veronii_BVH46 ['642.150.peg.2473'] ['642.150.peg.2475']
A_veronii_BAQ116 ['68.2853'] ['68.2855']
A_veronii_Hm21 ['99.3454'] ['99.3455']
